In [3]:
import numpy as np
import pandas as pd
import sys 
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel
from pandas import DataFrame
from bson import ObjectId 

In [4]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../..")))
from app.db.session import news_collection

ModuleNotFoundError: No module named 'app'

In [ ]:
cursor=news_collection.find({})
listed=list(cursor)
df=DataFrame(listed)
df.head()

,_id,title,content,image,author,link,category,published_at
0,68e24772c61b7a78edbf604b,“Би Ерөнхийлөгчтэй байнгын холбоотой ажилладаг...,УИХ-ын дарга Д.Амарбаясгалан сэтгүүлчдэд мэдээ...,https://mgl.gogo.mn/newsn/thumbnail/72/images/...,Г.Тэгшсүрэн,https://gogo.mn/r/n6l33y,Улс төр,2025-10-03
1,68e24776c61b7a78edbf604c,Д.Батлут: Чуулганы хуралдаандаа суухаар иртэл ...,УИХ-ын чуулганы хуралдаан долоо хоног бүрийн п...,https://mgl.gogo.mn/newsn/thumbnail/72/images/...,Г.Тэгшсүрэн,https://gogo.mn/r/4m0oon,Улс төр,2025-10-03
2,68e24779c61b7a78edbf604d,Туркийн Үндэсний Батлан хамгаалахын сайд Монго...,Турк Улсын Үндэсний Батлан хамгаалахын сайд Яш...,https://mgl.gogo.mn/newsn/thumbnail/72/images/...,А.Номин,https://gogo.mn/r/j0eww6,Улс төр,2025-10-03
3,68e2477dc61b7a78edbf604e,"Канад, Ираны Элчин сайд нар Итгэмжлэх жуух бич...","Ерөнхийлөгч У.Хүрэлсүхэд Канад, Иран улсын Элч...",https://mgl.gogo.mn/newsn/thumbnail/1000/image...,А.Номин,https://gogo.mn/r/gej115,Улс төр,2025-10-03
4,68e24781c61b7a78edbf604f,Б.Батцэцэг: Д.Амарбаясгалан дүүдээ хэлэхэд алб...,УИХ-ын гишүүн Б.Батцэцэг цаг үеийн асуудлаар м...,https://mgl.gogo.mn/newsn/thumbnail/72/np/2025...,Г.Тэгшсүрэн,https://gogo.mn/r/yk5llg,Улс төр,2025-10-03


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   _id           133 non-null    object        
 1   title         133 non-null    object        
 2   content       133 non-null    object        
 3   image         90 non-null     object        
 4   author        132 non-null    object        
 5   link          133 non-null    object        
 6   category      133 non-null    object        
 7   published_at  126 non-null    datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 8.4+ KB


In [ ]:
df['_id'] = df['_id'].astype(str)

In [ ]:
df.isnull().sum()

_id              0
title            0
content          0
image           43
author           1
link             0
category         0
published_at     7
dtype: int64

In [ ]:
df.drop(['image','link'],axis=1,inplace=True)
print(df.isna().sum())

_id             0
title           0
content         0
author          1
category        0
published_at    7
dtype: int64


In [ ]:
df["author"].fillna(df["author"].mode()[0],inplace=True)
print(df.isna().sum())

_id             0
title           0
content         0
author          0
category        0
published_at    7
dtype: int64


/var/folders/vb/7b1kw_mn7672zy_wxnktghz80000gn/T/ipykernel_3109/3544272394.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["author"].fillna(df["author"].mode()[0],inplace=True)


In [ ]:
df["published_at"].fillna(df["published_at"].median(),inplace=True)
print(df.isna().sum())

_id             0
title           0
content         0
author          0
category        0
published_at    0
dtype: int64


/var/folders/vb/7b1kw_mn7672zy_wxnktghz80000gn/T/ipykernel_3109/402516565.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["published_at"].fillna(df["published_at"].median(),inplace=True)


In [ ]:
tfidf = TfidfVectorizer(lowercase=True,analyzer='word', max_features=5000)
tf_matrix = tfidf.fit_transform(df['title'] + " " + df['content'])

In [ ]:
cosine_sim = linear_kernel(tf_matrix, tf_matrix)
indices = pd.Series(df.index, index=df['_id']).drop_duplicates()

In [ ]:
cosine_similarity = linear_kernel(tf_matrix, tf_matrix)
cosine_similarity

array([[1.        , 0.08046281, 0.03551675, ..., 0.08479254, 0.06676268,
        0.11309287],
       [0.08046281, 1.        , 0.17710583, ..., 0.05452715, 0.08992354,
        0.05079009],
       [0.03551675, 0.17710583, 1.        , ..., 0.08451423, 0.03464466,
        0.07421728],
       ...,
       [0.08479254, 0.05452715, 0.08451423, ..., 1.        , 0.10099244,
        0.18660658],
       [0.06676268, 0.08992354, 0.03464466, ..., 0.10099244, 1.        ,
        0.19098016],
       [0.11309287, 0.05079009, 0.07421728, ..., 0.18660658, 0.19098016,
        1.        ]], shape=(133, 133))

In [ ]:
def recommender(news_id):
    idx = indices[news_id]
    sim_scores = list(enumerate(cosine_similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return df["title"].iloc[movie_indices]

In [ ]:
recs = recommender(
    "68e24772c61b7a78edbf604b")

print("\n✅ Recommended news IDs:", recs[::5])



✅ Recommended news IDs: 4      Б.Батцэцэг: Д.Амарбаясгалан дүүдээ хэлэхэд алб...
132    Г.Амартүвшин: Төрийн өмчийн бодлого зохицуулал...
75                    БАГШ НАР: Бид ажилтай ядуус болсон
111    Цагдаа нар зочид буудалд гэнэтийн шалгалт хийх...
9      Д амин дэмээр баяжуулсан сүү ууснаар амьсгалын...
36     Монголын соёлыг чулуугаар илэрхийлсэн Торү Күр...
Name: title, dtype: object
